In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./EmployeeHandbook.pdf")
documents = loader.load()

print(len(documents))


/Users/mohamedrizwan/Desktop/RAG GIt/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


43


In [2]:

pdf_loader = loader.load()


In [3]:

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap= 200)

doc = text_splitter.split_documents(pdf_loader)



In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


/var/folders/gc/_6pc4cjj1_39_1622f88r3d00000gt/T/ipykernel_22200/3055314890.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [5]:
from langchain_community.vectorstores import FAISS
db1 = FAISS.from_documents(
    documents=doc[:50],
    embedding=embeddings
)

In [6]:
from langchain_community.vectorstores import Chroma

db2 = Chroma.from_documents(
    documents=doc,
    embedding=embeddings,
    persist_directory="./chroma_db"
)
db2.persist()

/var/folders/gc/_6pc4cjj1_39_1622f88r3d00000gt/T/ipykernel_22200/802160857.py:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db2.persist()


In [7]:
retriever1 = db1.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)


In [8]:
retriever2 = db2.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)


In [9]:
query = "What is leave policy?"

retrieved_docs = retriever1.invoke(query)

context1 = "\n\n".join(doc.page_content for doc in retrieved_docs)

context1

"excluding lunch and tea breaks.\nWork timings for teams are determined after considering the comfortable\nwork hours of all associated employees. Employees may choose their\nwork hours based on the team's working hours to facilitate better\ncollaboration.\nIf the project or team requires an employee to work on a Saturday,\nSunday, or public holiday, the employee is entitled to compensatory time\noff, which must be used within the next 3 months.\nWomen employees are encouraged to leave the office premises before 7\nPM. To do so, they must obtain prior approval from the functional heads\nand share their ride details with the admin team.\nHybrid Work Policy:\nPresidio India follows a hybrid work policy where employees with 2 or more\nyears of work experience are required to work from the office 3 days a\nweek, while employees with less than 2 years of work experience are\nrequired to work from the office all 5 days of the week. Please refer to the\nHybrid Work Policy document for more de

In [10]:
query = "What is leave policy?"

retrieved_docs = retriever2.invoke(query)

context2 = "\n\n".join(doc.page_content for doc in retrieved_docs)

context2


"37\nEmployee Separation\nEmployee Termination:\nIn the case of employee termination due to disciplinary action, the\nemployee will not be compensated.\nEmployment may be terminated for just cause and without notice after\nconsultation with the management team, for grave misconduct, including,\nbut not limited to, violation of the company code of conduct, failure to\nfollow company policy, breach of contract, violence or threatened\nviolence, threats or threatening behavior, stealing company money or\nproperty, falsifying records, extreme insubordination, and harassment.\nCompensation is at the discretion of the management.\nNotice Period:\nResignation by any employee shall be submitted in writing with a notice\nperiod of 30 days. After the notice of resignation has been given, the\nemployee may opt to revoke their resignation within a week's time.\nHowever, the acceptance of the resignation revocation is at the discretion\nof the management.\nExit Interview & Exit Feedback:\n\n37\nEmp

In [11]:
# Using Azure OpenAI LLM

In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    prompt="",
    azure_deployment="gpt-4.1",  
    api_version="",
    temperature=0,
    azure_endpoint="",
    api_key=""
)


/Users/mohamedrizwan/Desktop/RAG GIt/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3490: UserWarning: WARNING! prompt is not default parameter.
                prompt was transferred to model_kwargs.
                Please confirm that prompt is what you intended.
  if await self.run_code(code, result, async_=asy):


In [13]:
from langchain.prompts import PromptTemplate

rag_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an assistant answering questions strictly using the provided context.

If the answer is not present in the context, say:
"I don't know based on the provided context."

Context:
{context}

Question:
{question}

Answer:
"""
)


In [20]:
from langchain.chains import RetrievalQA

qa_chain1 = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever1,
    chain_type="stuff",# this is for multidocuments
    chain_type_kwargs={
        "prompt": rag_prompt
    },
    return_source_documents=True
)
qa_chain2 = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever2,
    chain_type="stuff",
    chain_type_kwargs={
        "prompt": rag_prompt
    },
    return_source_documents=True
)


In [30]:
query = "Who won the IPL 2025?"
result = qa_chain1(query)

print("Answer:\n", result["result"])


query = "How much can be reimbursed for certification?"
result = qa_chain1(query)

print("Answer:\n", result["result"])


Answer:
 I don't know based on the provided context. The employee handbook does not mention anything about the Indian Premier League (IPL) or its winners, including IPL 2025.
Answer:
 Based on the provided context, the answer is:

Upto INR 25,000/- per Quarter for Role Relevant Certifications.


In [29]:
query = "Who won the IPL 2025?"
result = qa_chain2(query)

print("Answer:\n", result["result"])


query = "How much can be reimbursed for certification?"
result = qa_chain2(query)

print("Answer:\n", result["result"])


Answer:
 I don't know based on the provided context.
Answer:
 According to the provided context, Certification: Upto INR 25,000/- per Quarter for Role Relevant Certifications.


In [ ]:
# Using Ollama LLM

In [23]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(
    model="llama3",
    temperature=0
)


/var/folders/gc/_6pc4cjj1_39_1622f88r3d00000gt/T/ipykernel_22200/2388233276.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [24]:
from langchain.chains import RetrievalQA

qa_chain1 = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever1,
    chain_type="stuff",# this is for multidocuments
    chain_type_kwargs={
        "prompt": rag_prompt
    },
    return_source_documents=True
)
qa_chain2 = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever2,
    chain_type="stuff",
    chain_type_kwargs={
        "prompt": rag_prompt
    },
    return_source_documents=True
)


In [26]:
query = "Who won the IPL 2025?"
result = qa_chain1(query)

print("Answer:\n", result["result"])


query = "How much can be reimbursed for certification?"
result = qa_chain1(query)

print("Answer:\n", result["result"])


Answer:
 I don't know based on the provided context. The employee handbook does not mention anything about the Indian Premier League (IPL) or its winners, including IPL 2025.
Answer:
 Based on the provided context, the answer is:

Upto INR 25,000/- per Quarter for Role Relevant Certifications.


In [28]:
query = "Who won the IPL 2025?"
result = qa_chain2(query)

print("Answer:\n", result["result"])


query = "What is Langchain?" 
result = qa_chain2(query)

print("Answer:\n", result["result"])


Answer:
 I don't know based on the provided context.
Answer:
 I don't know based on the provided context.
